In [138]:
import collections
import pandas as pd

In [354]:
# 데이터 로드
df= pd.read_csv('clustering/클러스터링_데이터/클러스터링_용인.csv')
# 인덱스를 열로 변환
# df.reset_index(inplace=True)
# df=df.rename(columns={'index':'행정동코드'})
df = df.drop(columns=['Unnamed: 0'])
df.head()

,행정동코드,입출차비율,소매/유통,의료/건강,70세_이상_비율,kmeans_cluster
0,41590262,0.32,117191.0,31705.0,0.08,2
1,41590259,0.30,128485.0,38765.0,0.06,2
2,41590585,0.27,59116.0,49065.0,0.03,2
3,41590253,0.26,142705.0,33462.0,0.07,2
4,41590400,0.62,12056.0,265.0,0.19,1


In [355]:
df_od=pd.read_csv('OD_데이터/OD용인.csv')
df_od

,Unnamed: 0,O_ADMI_CD,D_ADMI_CD,CNT
0,248,41590360,41590259,10450.06
1,64,41590259,41590360,10178.84
2,1,41590253,41590259,7181.29
3,413,41590530,41590520,7053.97
4,56,41590259,41590253,7033.10
...,...,...,...,...
767,326,41590400,41590630,3.13
768,621,41590588,41590320,3.06
769,369,41590515,41590550,2.89
770,215,41590340,41590600,2.87


In [356]:
####지역별 수정해야할 코드
#선택한 군집 필터링
##용인
df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==1)]
# #광명
# df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==0)]
#부천
# df=df[(df['kmeans_cluster']==0) | (df['kmeans_cluster']==1)]
# #수원 
# df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==1)]
# #시흥
# df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==1)]
# #안산
# df=df[(df['kmeans_cluster']==1) | (df['kmeans_cluster']==2)]
# #안양 
# df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==1)]
# #포천
# df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==1)]
# #하남
# df=df[(df['kmeans_cluster']==0) | (df['kmeans_cluster']==1)]
# #화성
# df=df[(df['kmeans_cluster']==2) | (df['kmeans_cluster']==0)]

df_area=df['행정동코드'].to_list()
df_area

[41590262,
 41590259,
 41590585,
 41590253,
 41590630,
 41590515,
 41590560,
 41590620,
 41590570,
 41590588,
 41590610,
 41590590,
 41590600,
 41590586,
 41590550,
 41590540,
 41590587,
 41590520,
 41590530]

In [357]:
df_freq = df_od[(df_od['O_ADMI_CD'].isin(df_area)) & (df_od['D_ADMI_CD'].isin(df_area))]
# df_freq=df_freq.reset_index(drop=True)
df_freq = df_freq.drop(columns=['Unnamed: 0'])
df_freq

,O_ADMI_CD,D_ADMI_CD,CNT
2,41590253,41590259,7181.29
3,41590530,41590520,7053.97
4,41590259,41590253,7033.10
5,41590520,41590530,6926.11
6,41590560,41590253,6879.51
...,...,...,...
745,41590515,41590570,7.00
751,41590515,41590520,4.50
752,41590520,41590515,4.50
755,41590590,41590515,4.50


In [358]:
df_freq.describe().T

,count,mean,std,min,25%,50%,75%,max
O_ADMI_CD,323.0,4.159052e+07,122.445604,41590253.00,41590530.00,41590570.00,41590590.00,41590630.00
D_ADMI_CD,323.0,4.159052e+07,122.269323,41590253.00,41590525.00,41590570.00,41590590.00,41590630.00
CNT,323.0,1.038362e+03,1487.748194,2.89,129.23,413.99,1354.45,7181.29


In [359]:
#행정동(노드),경로(간선)의 개수
df_o=df_freq['O_ADMI_CD'].value_counts()
df_o= df_o.reset_index()
df_o.columns = ['O_ADMI_CD','count']
df_d=df_freq['D_ADMI_CD'].value_counts()
df_d= df_d.reset_index()
df_d.columns = ['D_ADMI_CD','count']

In [360]:
df_count=pd.merge(df_o,df_d,how='outer',left_on='O_ADMI_CD', right_on='D_ADMI_CD')
node=len(df_count.index)
node #노드개수

19

In [361]:
line=len(df_freq.index)
line#간선개수

323

In [362]:
#탐색순서대로 고유값 id 부여 (1~n)
area_id={}
for i in range(node):
  df_freq=df_freq.replace(to_replace=df_count['O_ADMI_CD'].loc[i],value=i+1)
  area_id[i+1]=df_count['O_ADMI_CD'].loc[i]
df_freq #전처리 완료 데이터!

,O_ADMI_CD,D_ADMI_CD,CNT
2,9,6,7181.29
3,18,3,7053.97
4,6,9,7033.10
5,3,18,6926.11
6,15,9,6879.51
...,...,...,...
745,19,4,7.00
751,19,3,4.50
752,3,19,4.50
755,2,19,4.50


In [363]:
area_id #행정동별 id

{1: 41590262,
 2: 41590590,
 3: 41590520,
 4: 41590570,
 5: 41590585,
 6: 41590259,
 7: 41590587,
 8: 41590600,
 9: 41590253,
 10: 41590588,
 11: 41590610,
 12: 41590586,
 13: 41590620,
 14: 41590630,
 15: 41590560,
 16: 41590550,
 17: 41590540,
 18: 41590530,
 19: 41590515}

In [364]:
df_freq = df_freq.reset_index(drop = True)

# SCC 구현

In [365]:
v= node
e= line

graph = collections.defaultdict(list)
for i in range(e):
    a=df_freq['O_ADMI_CD'].loc[i]
    b=df_freq['D_ADMI_CD'].loc[i]
    
    if a > 0 and b > 0 :
        graph[a-1].append(b-1)

d = [-1 for _ in range(v)]
stack=[]
on_stack=[False for _ in range(v)]
id=0

def dfs(cur):
    global id
    id +=1
    d[cur] = id
    stack.append(cur)
    on_stack[cur] = True

    parent = d[cur]
    
    if cur in graph :
        for next in graph[cur]:
            if d[next] == -1:
                parent = min(parent, dfs(next))
            elif on_stack[next] :
                parent = min(parent, d[next])
    
    for next in graph[cur]:
        if d[next]==-1: #방문 이력이 없는 노드
            parent= min(parent, dfs(next))
        elif on_stack[next]: #방문 체크는 되어있지만 아직 처리 완료 X
            parent = min(parent, d[next])

    if parent == d[cur]: #자신과 부모가 동일
        global scc
        scc = []
        while True:
            node = stack.pop()
            on_stack[node] = False
            scc.append(node+1) #인덱스를 다시 숫자로 변환
            if cur == node:
                break
        print("Strongly Connected Component", *scc)
    return parent

for i in range(v):
    if d[i] ==-1: #방문 이력이 없는 노드
        dfs(i)

Strongly Connected Component 19 15 16 17 14 13 11 10 2 8 12 5 7 3 18 4 6 9 1


In [366]:
route=scc[::-1]
route

[1, 9, 6, 4, 18, 3, 7, 5, 12, 8, 2, 10, 11, 13, 14, 17, 16, 15, 19]

In [367]:
#노선 결정
df_route=pd.DataFrame({'행정동':route})
df_route['행정동']=df_route['행정동'].map(area_id)
df_route

,행정동
0,41590262
1,41590253
2,41590259
3,41590570
4,41590530
5,41590520
6,41590587
7,41590585
8,41590586
9,41590600


#시각화

In [368]:
#지역별 행정복지 위치 정보 필요 코드
df_care=pd.read_csv('OD_데이터/WellCareLongLat.csv')
df_care

,Unnamed: 0,행정동코드,Longitude,Latitude
0,0,4111156000,126.997406,37.310264
1,1,4111156000,126.997406,37.310264
2,2,4111156600,126.971373,37.297539
3,3,4111156600,126.971373,37.297539
4,4,4111157100,126.991132,37.302206
...,...,...,...,...
731,731,4129054000,126.998478,37.433454
732,732,4129055000,126.995408,37.448387
733,733,4129055000,126.995408,37.448387
734,734,4129056000,127.001863,37.426397


In [369]:
df_care.drop(columns=['Unnamed: 0'],inplace=True)

In [370]:
df_care.head()

,행정동코드,Longitude,Latitude
0,4111156000,126.997406,37.310264
1,4111156000,126.997406,37.310264
2,4111156600,126.971373,37.297539
3,4111156600,126.971373,37.297539
4,4111157100,126.991132,37.302206


In [371]:
df_pos = df_care.copy()

In [372]:
df_pos = df_pos.rename(columns={'행정동코드':'행정동'})

In [373]:
# 행정동코드에서 맨 뒤 두 자리를 제거
df_pos['행정동'] = df_pos['행정동'].astype(str).str[:-2]

# 결과 확인
print(df_pos)

          행정동   Longitude   Latitude
0    41111560  126.997406  37.310264
1    41111560  126.997406  37.310264
2    41111566  126.971373  37.297539
3    41111566  126.971373  37.297539
4    41111571  126.991132  37.302206
..        ...         ...        ...
731  41290540  126.998478  37.433454
732  41290550  126.995408  37.448387
733  41290550  126.995408  37.448387
734  41290560  127.001863  37.426397
735  41290560  127.001863  37.426397

[736 rows x 3 columns]


In [374]:
df_pos['행정동'] = df_pos['행정동'].astype(int) # 정수형으로 변환

In [375]:
df_pos

,행정동,Longitude,Latitude
0,41111560,126.997406,37.310264
1,41111560,126.997406,37.310264
2,41111566,126.971373,37.297539
3,41111566,126.971373,37.297539
4,41111571,126.991132,37.302206
...,...,...,...
731,41290540,126.998478,37.433454
732,41290550,126.995408,37.448387
733,41290550,126.995408,37.448387
734,41290560,127.001863,37.426397


In [376]:
df_route,df_pos

(         행정동
 0   41590262
 1   41590253
 2   41590259
 3   41590570
 4   41590530
 5   41590520
 6   41590587
 7   41590585
 8   41590586
 9   41590600
 10  41590590
 11  41590588
 12  41590610
 13  41590620
 14  41590630
 15  41590540
 16  41590550
 17  41590560
 18  41590515,           행정동   Longitude   Latitude
 0    41111560  126.997406  37.310264
 1    41111560  126.997406  37.310264
 2    41111566  126.971373  37.297539
 3    41111566  126.971373  37.297539
 4    41111571  126.991132  37.302206
 ..        ...         ...        ...
 731  41290540  126.998478  37.433454
 732  41290550  126.995408  37.448387
 733  41290550  126.995408  37.448387
 734  41290560  127.001863  37.426397
 735  41290560  127.001863  37.426397
 
 [736 rows x 3 columns])

In [377]:
# 행정동코드와 행정복지센터위치 연결
df_r = pd.merge(left = df_route,right = df_pos, how='inner',on='행정동')
df_r

,행정동,Longitude,Latitude
0,41590570,127.014617,37.206030
1,41590570,127.014617,37.206030
2,41590570,127.014617,37.206030
3,41590530,127.037242,37.206860
4,41590520,127.034976,37.213336
5,41590520,127.034976,37.213336
6,41590520,127.034976,37.213336
7,41590520,127.034976,37.213336
8,41590587,127.058678,37.209959
9,41590585,127.071950,37.206462


In [378]:
df_r.to_csv('정류장노선/행정복지관정보_용인.csv')
df_route.to_csv('정류장노선/정류장정보_용인.csv')

In [ ]:
## 지역별로 확인한 코드